In [1]:
from paddleocr import PaddleOCR,draw_ocr
from PIL import Image
from IPython.display import display

In [2]:
ocr = PaddleOCR(use_angle_cls=True, lang='en', show_log = False)
# EDIT ME: change the value of img_path to your file's location.
img_path = 'data/1996-393-011.pdf'
result = ocr.ocr(img_path, cls=True)
ocr_text = ''
for res in result:
    for line in res:
        ocr_text += (line[-1][0] + '\n')

print(ocr_text)

CLINTON
Maps and Gov Info
RESERV
Campaign Literature
GORE96
CAMPAIGNS,PQLITICAL-1996
PRESIDENT - CLINTQN -GENERAL
CLINTON ADMINISTRATION ACCOMPLISHMENTS
MEETING AMERICA'S CHALLENGES
Challenge:
To Cherish Our Children and Strengthen Qur Families
Signed the Family and Medical Leave Act (Feb.5, 1993) to enable workers to take up to 12 weeks of
unpaid leave to care for a family member without fear of losing their jobs.
Expanded the Earned Income Tax Credit (EITC) to provide tax relief for 15 million working families.
Reached agreements with the media and entertainment industry to develop a television ratings system and
to add three hours per week of educational programming.
Signed the Telecommunications Act of 1996 (Feb.8, 1996) that requires the V-chip in new televisions.
Collected a record S11 billion in child support in 1995-an almost 40% increase over 1992.
Proposed targeted measures to cut off children's access to tobacco and reduce their appeal to children.
Established a Childhood Im

In [3]:
from refined.data_types.base_types import Span, Entity
from refined.inference.processor import Refined

In [4]:
refined = Refined.from_pretrained(model_name='wikipedia_model',
                                  entity_set='wikipedia',
                                  use_precomputed_descriptions=False)

In [5]:
text = "ocr_text"
spans = refined.process_text(text)

named_entities = []
for span in spans:
    print(f"{span}\n")
    if isinstance(span[1], Entity):
        url = construct_wikidata_url(span[1].wikidata_entity_id)
        name = span[0]
        named_entities.append((name, url))
        

def construct_wikidata_url(id):
    return "https://www.wikidata.org/wiki/" + id

In [6]:
# EDIT ME: If you have an OpenAI API key and want to use it, set it here.
# Otherwise, leave this as an empty string.
openai_api_key = ""

prompt = (
        "Below is the OCR text from a document from UCLA's Campaign Literature collection, "
        " along with a list of named entities known to be associated with the document. "
        "Provide a Dublin Core metadata record for the original document. "
        "At a minumum, include 'Title', 'Subject', and 'Description' fields. "
        "Use 'sentence case' for the 'Title' field. "
        "Return only Dublin Core fields, correctly formatted as JSON. "
        f"The named entities are enclosed in three single quotes below: '''{spans}'''"
        f"The text from the document is enclosed in three single quotes below: '''{ocr_text}'''"
    )

from pprint import pprint

In [8]:
def get_gpt_as_dict(client, prompt, temperature = 0.1, top_p = 0.3) -> dict:
    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
        ],
        temperature=temperature,
        top_p=top_p,
    )
    response = completion.choices[0].message.content
    return json.loads(response)

if openai_api_key:
    from openai import OpenAI
    client = OpenAI(api_key=openai_api_key)
    response = get_gpt_as_dict(client, prompt)
    pprint(response)

else:
    print("Copy and paste the below prompt into your chosen LLM:")
    print(prompt)
    


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}